In [143]:
from tweepy import API 
from tweepy import Cursor
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

import numpy as np
import pandas as pd
import nltk
from snownlp import SnowNLP
from textblob import TextBlob
import matplotlib.pyplot as plt
import re


# Variables that contains the user credentials to access Twitter API 
CONSUMER_KEY    = '1UWXwx6ZGkwKbf5glOaVcayKn'
CONSUMER_SECRET = 'PCiY4A7LvU7FajeErB91yNlNdJzvygJYMTz9xuhW03RiFZLuxV'
# Access:
ACCESS_TOKEN  = '1156683457537482753-QrOn1F83Yr27fikbcyUbieCYwGoprK'
ACCESS_TOKEN_SECRET = 'Uo9N8VrtgixCPIGFFQq1jKv0VDMwVXXKOL5nQGsQrh2Cb'


# # # # TWITTER CLIENT # # # #
class TwitterClient():
    def __init__(self, twitter_user=None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth)

        self.twitter_user = twitter_user

    def get_twitter_client_api(self):
        return self.twitter_client

    def get_user_timeline_tweets(self, num_tweets):
        tweets = []
        for tweet in Cursor(self.twitter_client.user_timeline, id=self.twitter_user).items(num_tweets):
            tweets.append(tweet)
        return tweets

    def get_friend_list(self, num_friends):
        friend_list = []
        for friend in Cursor(self.twitter_client.friends, id=self.twitter_user).items(num_friends):
            friend_list.append(friend)
        return friend_list

    def get_home_timeline_tweets(self, num_tweets):
        home_timeline_tweets = []
        for tweet in Cursor(self.twitter_client.home_timeline, id=self.twitter_user).items(num_tweets):
            home_timeline_tweets.append(tweet)
        return home_timeline_tweets


# # # # TWITTER AUTHENTICATER # # # #
class TwitterAuthenticator():

    def authenticate_twitter_app(self):
        auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
        auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
        return auth

    
# # # # TWITTER STREAMER # # # #
class TwitterStreamer():
    """
    Class for streaming and processing live tweets.
    """
    def __init__(self):
        self.twitter_autenticator = TwitterAuthenticator()    

    def stream_tweets(self, fetched_tweets_filename, hash_tag_list):
        # This handles Twitter authetification and the connection to Twitter Streaming API
        listener = TwitterListener(fetched_tweets_filename)
        auth = self.twitter_autenticator.authenticate_twitter_app() 
        stream = Stream(auth, listener)

        # This line filter Twitter Streams to capture data by the keywords: 
        stream.filter(track=hash_tag_list)


# # # # TWITTER STREAM LISTENER # # # #
class TwitterListener(StreamListener):
    """
    This is a basic listener that just prints received tweets to stdout.
    """
    def __init__(self, fetched_tweets_filename):
        self.fetched_tweets_filename = fetched_tweets_filename

    def on_data(self, data):
        try:
            print(data)
            with open(self.fetched_tweets_filename, 'a') as tf:
                tf.write(data)
            return True
        except BaseException as e:
            print("Error on_data %s" % str(e))
        return True
          
    def on_error(self, status):
        if status == 420:
            # Returning False on_data method in case rate limit occurs.
            return False
        print(status)


class TweetAnalyzer():
    """
    Functionality for analyzing and categorizing content from tweets.
    """
    def tweets_to_data_frame(self, tweets):
        df = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweets'])

        df['id'] = np.array([tweet.id for tweet in tweets])
        df['len'] = np.array([len(tweet.text) for tweet in tweets])
        df['date'] = np.array([tweet.created_at for tweet in tweets])
        df['source'] = np.array([tweet.source for tweet in tweets])
        df['likes'] = np.array([tweet.favorite_count for tweet in tweets])
        df['retweets'] = np.array([tweet.retweet_count for tweet in tweets])
        df['Tweets'] = df.Tweets.apply(self.clean_tweet)

        return df
    
    def clean_tweet(self, tweet):
        stop_words = nltk.corpus.stopwords.words('english')
        lemmatizer = nltk.stem.WordNetLemmatizer()
        words = re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split()
        # kick out stop words
        words = [w for w in words if w not in stop_words]
        # word root
        words = [lemmatizer.lemmatize(word) for word in words]
        return ' '.join(words)
    
    def find_key_word(self, tweet):
        if key_word in tweet:
            return 1
        return 0

    def analyze_sentiment(self, tweet):
        analysis = TextBlob(tweet)
        
        if analysis.sentiment.polarity > 0.2:
            return 1
        elif analysis.sentiment.polarity < -0.2:
            return -1
        else:
            return 0

In [105]:
#hash_tag_list = ["donal trump", "hillary clinton", "barack obama", "bernie sanders"]
#fetched_tweets_filename = "tweets.txt"

#twitter_client = TwitterClient('pycon')
#print(twitter_client.get_user_timeline_tweets(1))

#    twitter_streamer = TwitterStreamer()
#    twitter_streamer.stream_tweets(fetched_tweets_filename, hash_tag_list)

In [144]:
twitter_client = TwitterClient()
tweet_analyzer = TweetAnalyzer()
api = twitter_client.get_twitter_client_api()

tweets = api.user_timeline(screen_name="realDonaldTrump", count=200,page=1)
df = tweet_analyzer.tweets_to_data_frame(tweets)
for i in range(2,13):
    tweets = api.user_timeline(screen_name="realDonaldTrump", count=200,page=i)
    df = pd.concat([df,tweet_analyzer.tweets_to_data_frame(tweets)])
df = df.reset_index()
df['sentiment'] = df['Tweets'].apply(tweet_analyzer.analyze_sentiment)

df.head(5)

,index,Tweets,id,len,date,source,likes,retweets,sentiment
0,0,Highest Trump Poll Numbers EVER Thank,1.230890e+18,54.0,2020-02-21 16:19:00,Twitter for iPhone,3360.0,1154.0,0
1,0,Great State Texas incredible supporter MAGA Ag...,1.228439e+18,140.0,2020-02-14 22:01:09,Twitter for iPhone,29521.0,8724.0,1
2,1,Brooks running Congress Great State Alabama He...,1.228439e+18,140.0,2020-02-14 22:00:27,Twitter for iPhone,26429.0,7757.0,1
3,2,The Greatest name politics Patrick Great State...,1.228438e+18,140.0,2020-02-14 21:58:08,Twitter for iPhone,26916.0,7824.0,1
4,3,Thank beautiful country Cambodia accepting shi...,1.228419e+18,140.0,2020-02-14 20:41:01,Twitter for iPhone,65156.0,13913.0,1


In [146]:
key_word = 'China'
df['include_China'] = df['Tweets'].apply(tweet_analyzer.find_key_word)
df.describe()

,index,id,len,likes,retweets,sentiment,include_China
count,1396.000000,1.396000e+03,1396.000000,1396.000000,1396.000000,1396.000000,1396.000000
mean,99.073782,1.217742e+18,117.452722,46347.549427,17361.901146,0.156160,0.012178
std,57.592633,6.602854e+15,37.563193,78482.001451,15967.842497,0.604616,0.109718
min,0.000000,1.207294e+18,13.000000,0.000000,1154.000000,-1.000000,0.000000
25%,49.000000,1.212066e+18,104.000000,0.000000,7753.250000,0.000000,0.000000
50%,99.000000,1.218160e+18,140.000000,0.000000,12990.000000,0.000000,0.000000
75%,149.000000,1.224704e+18,140.000000,77741.500000,22409.250000,1.000000,0.000000
max,199.000000,1.230890e+18,148.000000,805715.000000,169865.000000,1.000000,1.000000


In [226]:
d1 = df.loc[df.include_China == 1]
d1 = d1.loc[:,['date','sentiment']]
d1.date = pd.to_datetime(d1.date.apply(lambda x:str(x)[:10]))
d1.head()

,date,sentiment
284,2020-02-07,1
394,2020-02-03,1
435,2020-01-24,0
509,2020-01-22,1
718,2020-01-16,1


In [227]:
from pandas_datareader import data as pdr
import yfinance as yf
from scipy.stats import f
yf.pdr_override()
p1 = pd.DataFrame(pdr.get_data_yahoo('000001.SS', start="2019-12-01", end="2020-02-20")['Adj Close'])
p1['ret'] = p1['Adj Close']/p1['Adj Close'].shift(1) - 1
p1['class'] = p1.ret.apply(lambda x:1 if x > 0.003 else -1 if x < -0.003 else 0)
p1 = p1.dropna()
p1.head()

[*********************100%***********************]  1 of 1 completed


,Adj Close,ret,class
Date,,,
2019-12-03,2884.699951,0.003091,1
2019-12-04,2878.115967,-0.002282,0
2019-12-05,2899.468994,0.007419,1
2019-12-06,2912.013916,0.004327,1
2019-12-09,2914.478027,0.000846,0


In [228]:
tem = []
for i in d1.date:
    if i not in list(p1.index):
        print(i)
        tem.append('x')
    else:
        tem.append(list(p1.loc[p1.index==i]['class'])[0])
d1['result'] = tem
d1['check'] = d1.sentiment==d1.result
print(sum(d1.check))
d1

2020-01-24 00:00:00
2020-01-01 00:00:00
3


,date,sentiment,result,check
284,2020-02-07,1,1,True
394,2020-02-03,1,-1,False
435,2020-01-24,0,x,False
509,2020-01-22,1,0,False
718,2020-01-16,1,-1,False
720,2020-01-16,1,-1,False
722,2020-01-16,0,-1,False
723,2020-01-16,1,-1,False
724,2020-01-16,0,-1,False
725,2020-01-16,0,-1,False


In [3]:
twitter_client = TwitterClient()
tweet_analyzer = TweetAnalyzer()

api = twitter_client.get_twitter_client_api()

tweets = api.user_timeline(screen_name="realDonaldTrump", count=20)

#print(dir(tweets[0]))
#print(tweets[0].retweet_count)

df = tweet_analyzer.tweets_to_data_frame(tweets)

# Get average length over all tweets:
print(np.mean(df['len']))

# Get the number of likes for the most liked tweet:
print(np.max(df['likes']))

# Get the number of retweets for the most retweeted tweet:
print(np.max(df['retweets']))

#print(df.head(10))

# Time Series
time_likes = pd.Series(data=df['len'].values, index=df['date'])
time_likes.plot(figsize=(16, 4), color='r')
plt.show()

time_favs = pd.Series(data=df['likes'].values, index=df['date'])
time_favs.plot(figsize=(16, 4), color='r')
plt.show()

time_retweets = pd.Series(data=df['retweets'].values, index=df['date'])
time_retweets.plot(figsize=(16, 4), color='r')
plt.show()

# Layered Time Series:
time_likes = pd.Series(data=df['likes'].values, index=df['date'])
time_likes.plot(figsize=(16, 4), label="likes", legend=True)

time_retweets = pd.Series(data=df['retweets'].values, index=df['date'])
time_retweets.plot(figsize=(16, 4), label="retweets", legend=True)
plt.show()

122.9
98761
29387
